In [1]:
import pandas as pd

In [2]:
file = "D:\\My Personal Documents\\Learnings\\Data Science\\Data Sets\\Twiter Sentiment Analysis\\train_E6oV3lV.csv"
test = 'D:\\My Personal Documents\\Learnings\\Data Science\\Data Sets\\Twiter Sentiment Analysis\\test_tweets_anuFYb8.csv'
tweets = pd.read_csv(file)
test_tweets=pd.read_csv(test)

In [3]:
tweets1=tweets[tweets.label==1]

In [4]:
tweets=tweets.append(tweets1)
tweets=tweets.append(tweets1)
tweets=tweets.append(tweets1)

In [5]:
label=tweets.label

In [6]:
tweets[tweets.label==1].count()

id       8968
label    8968
tweet    8968
dtype: int64

In [7]:
tweets=tweets.append(test_tweets)

C:\Amrendra\Anaconda\lib\site-packages\pandas\core\frame.py:6201: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=True'.

To retain the current behavior and silence the warning, pass sort=False

  sort=sort)


In [8]:
tweets.groupby(tweets.label).tweet.count()

label
0.0    29720
1.0     8968
Name: tweet, dtype: int64

In [9]:
cleaned_tweet = tweets.tweet.str.replace("[^a-zA-Z]", " ")

In [10]:
cleaned_tweet=cleaned_tweet.apply(lambda x :" ".join(w for w in x.split() if w != 'user' ))

In [11]:
from nltk.stem.porter import PorterStemmer
#from nltk.stem.snowball import SnowballStemmer

In [12]:
stem=PorterStemmer()

In [13]:
cleaned_tweet = cleaned_tweet.apply(lambda x : " ".join( stem.stem(w) for w in x.split()))

In [14]:
cleaned_tweet = cleaned_tweet.apply(lambda x : " ".join( w for w in x.split()))

In [15]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [16]:
tfidf = TfidfVectorizer(max_features=9300,stop_words='english')

In [17]:
bow=tfidf.fit_transform(cleaned_tweet)

In [18]:
train_bow = bow[:38688,:]
test_bow = bow[38688:,:]

In [19]:
from sklearn.model_selection import train_test_split

In [20]:
x_train,x_test,y_train, y_test = train_test_split(train_bow,label,test_size=0.2,random_state=3, stratify=label)

In [21]:
print(x_train.shape)
print(y_train.shape)
print(x_test.shape)
print(y_test.shape)

(30950, 9300)
(30950,)
(7738, 9300)
(7738,)


In [22]:
x_train[0][0]

<1x9300 sparse matrix of type '<class 'numpy.float64'>'
	with 7 stored elements in Compressed Sparse Row format>

In [23]:
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.optimizers import SGD
from keras.datasets import mnist
from keras.callbacks import ModelCheckpoint

C:\Amrendra\Anaconda\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [24]:
#n_class=2
#y_train = keras.utils.to_categorical(y_train,n_class)
#y_test = keras.utils.to_categorical(y_test,n_class)

In [25]:
import os
output_dir = 'model_tweet_output1/dense'
modelcheckpoint = ModelCheckpoint(filepath=output_dir+"/weights.{epoch:02d}.hdf5")
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

In [26]:
model = Sequential()
model.add(Dense(3000,activation='relu', input_shape=(9300,)))
model.add(Dropout(0.2))
#model.add(Dense(2000,activation='relu', input_shape=(9300,)))
#model.add(Dropout(0.3))
model.add(Dense(1,activation='sigmoid'))
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [27]:
model.fit(x_train,y_train,epochs=5,verbose=1, validation_data=(x_test,y_test),callbacks=[modelcheckpoint])

Train on 30950 samples, validate on 7738 samples
Epoch 1/5
30950/30950 [==============================] - 510s 16ms/step - loss: 0.1632 - acc: 0.9352 - val_loss: 0.0773 - val_acc: 0.9730
Epoch 2/5
30950/30950 [==============================] - 522s 17ms/step - loss: 0.0311 - acc: 0.9900 - val_loss: 0.0434 - val_acc: 0.9849
Epoch 3/5
30950/30950 [==============================] - 518s 17ms/step - loss: 0.0090 - acc: 0.9975 - val_loss: 0.0389 - val_acc: 0.9889
Epoch 4/5
30950/30950 [==============================] - 521s 17ms/step - loss: 0.0046 - acc: 0.9987 - val_loss: 0.0419 - val_acc: 0.9882
Epoch 5/5
30950/30950 [==============================] - 523s 17ms/step - loss: 0.0034 - acc: 0.9990 - val_loss: 0.0433 - val_acc: 0.9885


In [34]:
model.load_weights(output_dir+'/weights.03.hdf5')

In [35]:
y_hat=model.predict(test_bow)

In [36]:
y_hat = pd.DataFrame(y_hat, columns=['Prob']).Prob.apply(lambda x : 1 if x >=0.6 else 0)

In [37]:
y_hat.to_csv('D:\\My Personal Documents\\Learnings\\Data Science\\Data Sets\\Twiter Sentiment Analysis\\submit_TFIDF_MLP.csv')